# Imports and helper functions

To import packages and modules to Jupyter notebook, you need to setup a conda environment. Here we call it `gpst`.
```
conda create --name gpst
conda install -c conda-forge pypsa pandapower jupyterlab
pip install yaml vresutils==0.3.1
```
Upgrade to pandapower to develop branch
```
pip install git+git://github.com/e2nIEE/pandapower@develop
```
To  add the kernel for the jupyter notebook
```
pip install ipykernel
ipython kernel install --user --name=gpst
```

Open the jupyter lab notebook by typing `jupyter lab` in the terminal.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import timeit
import pandas as pd

import numpy as np
import pandapower as pp
import pandapower.converter
        
import logging

logger = logging.getLogger(__name__)

from helpers import _sets_path_to_root
from helpers import compare_to_matpower
from helpers import make_name_to_index
from helpers import pypsa_validation
from helpers import add_nice_carrier_names
from helpers import load_costs

# Show all pandas columns in jupyter
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

This is the repository path:  /home/max/OneDrive/PHD-Flexibility/07_pypsa-africa/0github/power-flow-exercise
Had to go 1 folder(s) up.


In [3]:
# Optional. Take local PyPSA-Dev installation to make adjustments
print(os.getcwd())
pypsa_path = os.getcwd()+"/example-pypsa/PyPSA"  # require to have `PyPSA` cloned in ~/power-flow-exercise/example_pypsa/<PyPSA>`
print(pypsa_path)
import sys
sys.path.insert(0, f"{pypsa_path}")

/home/max/OneDrive/PHD-Flexibility/07_pypsa-africa/0github/power-flow-exercise
/home/max/OneDrive/PHD-Flexibility/07_pypsa-africa/0github/power-flow-exercise/example-pypsa/PyPSA


In [4]:
import pypsa

# Pandapower import of Matpower

In [5]:
net = pp.converter.from_mpc(os.path.join("reference-matpower", "RTS_GMLC.mat"))

In [6]:
net.sgen.drop(net.sgen[~net.sgen.in_service].index, inplace=True)  # TODO: check if this is correct (dropping not in_service elements from net.sgen
pp.runpp(net)
compare_to_matpower(net)  # Info: Will be saved in logger 'log.out'
make_name_to_index(net)

# PyPSA import of Pandapower network

In [7]:
# Build empty PyPSA network
network = pypsa.Network()
# Import pandapower
network.import_from_pandapower_net(net, True, use_pandapower_index=True)
pypsa_validation(network, net)
n = network

In [8]:
# Run powerflow
    #n.lopf()
    #n.pf(use_seed=True, distribute_slack=True)
    #n.pf(x_tol=1e-6,)
    # n.lpf()

# n.pf()


# Planning exercise

### Preparation

Load costs and data modification 

In [9]:
costs = load_costs(Nyears=1., tech_costs=None, config=None, elec_config=None)
add_nice_carrier_names(n)

In [10]:
n.generators.loc[:,"carrier"] = "OCGT"
n.generators.loc[:,"capital_cost"] = costs.loc["OCGT", "capital_cost"]
n.generators.loc[:,"marginal_cost"] = costs.loc["OCGT", "marginal_cost"]
n.generators.loc[:,"lifetime"] = costs.loc["OCGT", "lifetime"]
n.generators.loc[:,"p_nom_extendable"] = True


n.lines.loc[:,'capital_cost'] = (n.lines['length'] * 1 * costs.at['HVAC overhead', 'capital_cost'])
n.lines.loc[:,"s_nom_extendable"] = True


n.transformers.loc[:,"s_nom_extendable"] = True


#n.loads_t["p"] = n.loads_t["p"] * 3
n.loads["p_set"] = n.loads["p_set"] * 2

In [11]:
n.lines.loc["104", "r"] = 1.
n.lines.loc["104", "b"] = 0.000147

### Constraints

In [12]:
from pypsa.linopf import (get_var, define_constraints, linexpr, join_exprs,
                          network_lopf, ilopf)
import yaml

In [13]:
### Loads raw config.yaml
path = os.getcwd()+"/example-pypsa/config.yaml"
with open(path, "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
config = config

In [14]:
def fix_bus_production(n, snapshots):
    total_demand = n.loads_t.p.sum().sum()
    prod_per_bus = linexpr((1, get_var(n, 'Generator', 'p'))).groupby(n.generators.bus, axis=1).apply(join_exprs)
    define_constraints(n, prod_per_bus, '>=', str(total_demand/1000), 'Bus', 'production_share')
    print(total_demand)
    print(prod_per_bus)

def extra_functionality(n, snapshots):
    fix_bus_production(n, snapshots)

# Solve network

In [15]:
def solve_network(n, config, opts='', **kwargs):
    solver_options = config['solving']['solver'].copy()
    solver_name = solver_options.pop('name')
    cf_solving = config['solving']['options']
    track_iterations = cf_solving.get('track_iterations', False)
    min_iterations = cf_solving.get('min_iterations', 4)
    max_iterations = cf_solving.get('max_iterations', 6)

    # add to network for extra_functionality
    n.config = config
    n.opts = opts

    if cf_solving.get('skip_iterations', False):
        network_lopf(n, solver_name=solver_name, solver_options=solver_options,
                     extra_functionality=extra_functionality, **kwargs)
    else:
        ilopf(n, solver_name=solver_name, solver_options=solver_options,
              track_iterations=track_iterations,
              min_iterations=min_iterations,
              max_iterations=max_iterations,
              extra_functionality=extra_functionality, **kwargs)
    return n


In [16]:
from pathlib import Path

tmpdir = config['solving'].get('tmpdir')
if tmpdir is not None:
        Path(tmpdir).mkdir(parents=True, exist_ok=True)

%debug
n = solve_network(n, config=config, solver_dir=tmpdir,)

0.0
bus
0     +1.000000 x217\n+1.000000 x249\n+1.000000 x250...
1     +1.000000 x218\n+1.000000 x252\n+1.000000 x253...
12    +1.000000 x255\n+1.000000 x256\n+1.000000 x257...
13                                     +1.000000 x243\n
14     +1.000000 x220\n+1.000000 x258\n+1.000000 x259\n
15                                     +1.000000 x221\n
17                                     +1.000000 x222\n
20                                     +1.000000 x244\n
21    +1.000000 x245\n+1.000000 x294\n+1.000000 x295...
22    +1.000000 x223\n+1.000000 x260\n+1.000000 x261...
24    +1.000000 x224\n+1.000000 x264\n+1.000000 x265...
25    +1.000000 x225\n+1.000000 x266\n+1.000000 x267...
30                     +1.000000 x226\n+1.000000 x269\n
36     +1.000000 x227\n+1.000000 x270\n+1.000000 x271\n
37                                     +1.000000 x246\n
38    +1.000000 x228\n+1.000000 x272\n+1.000000 x300...
39                                     +1.000000 x229\n
41                                     +

# Inspect PyPSA network before solving

In [17]:
n.snapshots

Index(['now'], dtype='object', name='snapshot')

In [18]:
n.global_constraints

attribute,type,investment_period,carrier_attribute,sense,constant,mu
GlobalConstraint,,,,,,


In [19]:
n.buses

,v_nom,v_mag_pu_set,type,x,y,carrier,unit,v_mag_pu_min,v_mag_pu_max,control,sub_network,generator,production_share
Bus,,,,,,,,,,,,,
0,138.0,1.0468,,0.0,0.0,AC,None,0.0,inf,PV,0,0,-0.0
1,138.0,1.0467,,0.0,0.0,AC,None,0.0,inf,PV,0,1,-0.0
2,138.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,0,NaN,NaN
3,138.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,0,NaN,NaN
4,138.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,0,NaN,NaN
5,138.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,0,NaN,NaN
6,138.0,1.0500,,0.0,0.0,AC,None,0.0,inf,PV,0,2,-0.0
7,138.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,0,NaN,NaN
8,138.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,0,NaN,NaN


In [20]:
n.generators

,p_set,q_set,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,sign,carrier,marginal_cost,build_year,lifetime,capital_cost,efficiency,committable,start_up_cost,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,8.00,0.000000,0,PV,,0.0,True,0.0,inf,0.0,1.0,1.0,OCGT,58.384615,0,30.0,47234.561404,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.000000e+00
1,8.00,0.000000,1,PV,,0.0,True,0.0,inf,0.0,1.0,1.0,OCGT,58.384615,0,30.0,47234.561404,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.000000e+00
2,355.00,0.000000,6,PV,,0.0,True,0.0,inf,0.0,1.0,1.0,OCGT,58.384615,0,30.0,47234.561404,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,6.488669e+02
3,5.00,0.000000,14,PV,,0.0,True,0.0,inf,0.0,1.0,1.0,OCGT,58.384615,0,30.0,47234.561404,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.000000e+00
4,155.00,0.000000,15,PV,,0.0,True,0.0,inf,0.0,1.0,1.0,OCGT,58.384615,0,30.0,47234.561404,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,5.620000e+02
5,355.00,0.000000,17,PV,,0.0,True,0.0,inf,0.0,1.0,1.0,OCGT,58.384615,0,30.0,47234.561404,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,6.660000e+02
6,155.00,0.000000,22,PV,,0.0,True,0.0,inf,0.0,1.0,1.0,OCGT,58.384615,0,30.0,47234.561404,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,3.867384e+02
7,8.00,0.000000,24,PV,,0.0,True,0.0,inf,0.0,1.0,1.0,OCGT,58.384615,0,30.0,47234.561404,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,5.155769e+02
8,8.00,0.000000,25,PV,,0.0,True,0.0,inf,0.0,1.0,1.0,OCGT,58.384615,0,30.0,47234.561404,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,5.030507e+02


In [21]:
n.loads_t

{'p_set': Empty DataFrame
 Columns: []
 Index: [now],
 'q_set': Empty DataFrame
 Columns: []
 Index: [now],
 'p': Load          0      1      2      3      4      5      6      7      8  \
 snapshot                                                                  
 now       216.0  194.0  360.0  148.0  142.0  272.0  250.0  342.0  350.0   
 
 Load          9     10     11     12     13     14     15     16     17  \
 snapshot                                                                  
 now       390.0  530.0  388.0  634.0  200.0  666.0  362.0  256.0  216.0   
 
 Load         18     19     20     21     22     23     24     25     26  \
 snapshot                                                                  
 now       194.0  360.0  148.0  142.0  272.0  250.0  342.0  350.0  390.0   
 
 Load         27     28     29     30     31     32     33     34     35  \
 snapshot                                                                  
 now       530.0  388.0  634.0  200.0  666.0 

In [22]:
n.loads

,p_set,q_set,bus,carrier,type,sign
Load,,,,,,
0,216.0,22.0,0,,,-1.0
1,194.0,20.0,1,,,-1.0
2,360.0,37.0,2,,,-1.0
3,148.0,15.0,3,,,-1.0
4,142.0,14.0,4,,,-1.0
5,272.0,28.0,5,,,-1.0
6,250.0,25.0,6,,,-1.0
7,342.0,35.0,7,,,-1.0
8,350.0,36.0,8,,,-1.0


In [23]:
n.lines

,r,x,b,s_nom,bus0,bus1,length,num_parallel,type,g,s_nom_extendable,s_nom_min,s_nom_max,s_max_pu,capital_cost,build_year,lifetime,carrier,terrain_factor,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,v_nom
Line,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.57132,2.66616,0.002421,101.036297,0,1,1.0,1.0,,0.0,True,0.0,inf,1.0,38.003656,0,inf,AC,1.0,-inf,inf,0,0.00014,0.000030,0.0,46.1000,0.00014,0.000030,1.059305e-10,138.0
1,10.47420,40.18284,0.000299,101.036297,0,2,1.0,1.0,,0.0,True,0.0,inf,1.0,38.003656,0,inf,AC,1.0,-inf,inf,0,0.00211,0.000550,0.0,5.7000,0.00211,0.000550,1.034263e+02,138.0
2,4.18968,16.18740,0.000121,101.036297,0,4,1.0,1.0,,0.0,True,0.0,inf,1.0,38.003656,0,inf,AC,1.0,-inf,inf,0,0.00085,0.000220,0.0,2.3000,0.00085,0.000220,2.054936e+02,138.0
3,6.28452,24.18588,0.000179,101.036297,1,3,1.0,1.0,,0.0,True,0.0,inf,1.0,38.003656,0,inf,AC,1.0,-inf,inf,0,0.00127,0.000330,0.0,3.4000,0.00127,0.000330,1.480000e+02,138.0
4,9.52200,36.56448,0.000273,101.036297,1,5,1.0,1.0,,0.0,True,0.0,inf,1.0,38.003656,0,inf,AC,1.0,-inf,inf,0,0.00192,0.000500,0.0,5.2000,0.00192,0.000500,1.567051e+02,138.0
5,5.90364,22.66236,0.000168,101.036297,2,8,1.0,1.0,,0.0,True,0.0,inf,1.0,38.003656,0,inf,AC,1.0,-inf,inf,0,0.00119,0.000310,0.0,3.2000,0.00119,0.000310,2.543651e+01,138.0
6,5.14188,19.80576,0.000147,101.036297,3,8,1.0,1.0,,0.0,True,0.0,inf,1.0,38.003656,0,inf,AC,1.0,-inf,inf,0,0.00104,0.000270,0.0,2.8000,0.00104,0.000270,2.456633e-09,138.0
7,4.38012,16.75872,0.000126,101.036297,4,9,1.0,1.0,,0.0,True,0.0,inf,1.0,38.003656,0,inf,AC,1.0,-inf,inf,0,0.00088,0.000230,0.0,2.4000,0.00088,0.000230,6.349365e+01,138.0
8,2.66616,11.61684,0.012912,101.036297,5,9,1.0,1.0,,0.0,True,0.0,inf,1.0,38.003656,0,inf,AC,1.0,-inf,inf,0,0.00061,0.000140,0.0,245.9000,0.00061,0.000140,1.152949e+02,138.0


In [24]:
n.transformers

,phase_shift,s_nom,bus0,bus1,r,x,g,b,tap_position,type,model,s_nom_extendable,s_nom_min,s_nom_max,s_max_pu,capital_cost,num_parallel,tap_ratio,tap_side,build_year,lifetime,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Transformer,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.4,23,2,0.00002,0.00084,0.0,0.0,1,,t,True,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,0.0,0.0021,0.00005,231.137215
1,0.0,0.4,10,8,0.00002,0.00084,0.0,0.0,1,,t,True,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,0.0,0.0021,0.00005,213.914611
2,0.0,0.4,11,8,0.00002,0.00084,0.0,0.0,1,,t,True,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,0.0,0.0021,0.00005,187.330391
3,0.0,0.4,10,9,0.00002,0.00084,0.0,0.0,1,,t,True,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,0.0,0.0021,0.00005,234.192713
4,0.0,0.4,11,9,0.00002,0.00084,0.0,0.0,1,,t,True,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,0.0,0.0021,0.00005,207.608493
5,0.0,0.4,47,26,0.00002,0.00084,0.0,0.0,1,,t,True,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,0.0,0.0021,0.00005,191.994115
6,0.0,0.4,34,32,0.00002,0.00084,0.0,0.0,1,,t,True,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,0.0,0.0021,0.00005,186.204699
7,0.0,0.4,35,32,0.00002,0.00084,0.0,0.0,1,,t,True,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,0.0,0.0021,0.00005,161.397546
8,0.0,0.4,34,33,0.00002,0.00084,0.0,0.0,1,,t,True,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,0,0.0021,0.00005,0.0,0.0,0.0021,0.00005,211.718112


In [25]:
n.links

attribute,bus0,bus1,type,carrier,efficiency,build_year,lifetime,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_set,p_min_pu,p_max_pu,capital_cost,marginal_cost,length,terrain_factor,p_nom_opt
Link,,,,,,,,,,,,,,,,,,,


In [26]:
n.stores

attribute,bus,type,carrier,e_nom,e_nom_extendable,e_nom_min,e_nom_max,e_min_pu,e_max_pu,e_initial,e_initial_per_period,e_cyclic,e_cyclic_per_period,p_set,q_set,sign,marginal_cost,capital_cost,standing_loss,build_year,lifetime,e_nom_opt
Store,,,,,,,,,,,,,,,,,,,,,,


In [27]:
n.storage_units

attribute,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,q_set,sign,carrier,marginal_cost,capital_cost,build_year,lifetime,state_of_charge_initial,state_of_charge_initial_per_period,state_of_charge_set,cyclic_state_of_charge,cyclic_state_of_charge_per_period,max_hours,efficiency_store,efficiency_dispatch,standing_loss,inflow,p_nom_opt
StorageUnit,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [28]:
n.shunt_impedances

,bus,g,b,sign,g_pu,b_pu,v_nom
ShuntImpedance,,,,,,,
0,5,0.0,0.005251,-1.0,0.0,100.0,138.0
1,29,0.0,0.005251,-1.0,0.0,100.0,138.0
2,53,0.0,0.005251,-1.0,0.0,100.0,138.0


In [29]:
n.carriers

attribute,co2_emissions,color,nice_name,max_growth
Carrier,,,,


In [30]:
n.objective

808147084.4479713

# Planning Exercise Test

# Solve network